<a href="https://colab.research.google.com/github/sandervaneck/DataScience/blob/main/DataScienceAsignment_Course4_SanderVanEck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade scikit-learn==1.0.0
!pip install imbalanced-learn
!pip install treeinterpreter
!pip install dtreeviz
!pip install graphviz
!pip install opendatasets
!pip install pandas

In [56]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score, make_scorer, explained_variance_score, precision_score, mean_absolute_error, classification_report, RocCurveDisplay, confusion_matrix,  recall_score, accuracy_score, roc_auc_score, precision_recall_curve, RocCurveDisplay, f1_score, mean_squared_error
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split, learning_curve
import math
import seaborn
from sklearn.datasets import make_classification
from matplotlib import pyplot
import graphviz
from sklearn.tree import export_graphviz
import io
from google.colab import files
from sklearn import metrics, utils, preprocessing
import matplotlib.pyplot as plt
from matplotlib import cm as cm
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

%matplotlib inline
from sklearn.ensemble import IsolationForest

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import utils
import pydot
from IPython.display import Image
from imblearn.combine import SMOTETomek
from collections import Counter

from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV

In [3]:
uploaded = files.upload()
df_all = pd.read_csv(io.BytesIO(uploaded['resampled_df_05.csv']))

Saving resampled_df_05.csv to resampled_df_05.csv


Explore Dataset

In [ ]:
df_all.describe()

Check for missing data among arrivals

In [ ]:
numberOfNullArrivalDelayWhenArrivalData = sum((df_all.CRS_ARR_TIME >0) & (df_all.ARR_TIME >0) & (df_all.ARR_DELAY.isnull()))
numberOfNoDataAndNotCancelledNorDiverted = sum(((df_all.CANCELLED==0) & (df_all.DIVERTED==0)) & (df_all.ARR_TIME.isnull()))

print(f"Number of flights with no arrival delay data, but can be derived: {numberOfNullArrivalDelayWhenArrivalData}")
print(f"Number of flights that are missing data and were not cancelled, nor diverted: {numberOfNoDataAndNotCancelledNorDiverted}")

Data transformation on entire dataset

In [4]:
df_all['CANC_DIV'] = np.where(((df_all['CANCELLED'] == 1) | (df_all['DIVERTED'] == 1)), 1, 0)
df_all.loc[(df_all.CRS_ARR_TIME >0) & (df_all.ARR_TIME >0) & (df_all.ARR_DELAY.isnull()), "ARR_DELAY"] = df_all['CRS_ARR_TIME'] - df_all['ARR_TIME']

df_all['DATES'] =  pd.to_datetime(df_all.FL_DATE)

# Month variable
df_all['month'] = pd.to_datetime(df_all['DATES']).dt.month
# Day variable
df_all['day'] = pd.to_datetime(df_all['DATES']).dt.day
# Weekday variable
df_all['weekday'] = pd.to_datetime(df_all['DATES']).dt.weekday 

df_all['scheduled_hour'] = 0
df_all.loc[df_all['CRS_DEP_TIME'] < 100, 'scheduled_hour'] = 1
df_all.loc[(df_all['CRS_DEP_TIME'] >= 100) & (df_all['CRS_DEP_TIME'] < 200), 'scheduled_hour'] = 2
df_all.loc[(df_all['CRS_DEP_TIME'] >= 200) & (df_all['CRS_DEP_TIME'] < 300), 'scheduled_hour'] = 3
df_all.loc[(df_all['CRS_DEP_TIME'] >= 300) & (df_all['CRS_DEP_TIME'] < 400), 'scheduled_hour'] = 4
df_all.loc[(df_all['CRS_DEP_TIME'] >= 400) & (df_all['CRS_DEP_TIME'] < 500), 'scheduled_hour'] = 5
df_all.loc[(df_all['CRS_DEP_TIME'] >= 500) & (df_all['CRS_DEP_TIME'] < 600), 'scheduled_hour'] = 6
df_all.loc[(df_all['CRS_DEP_TIME'] >= 600) & (df_all['CRS_DEP_TIME'] < 700), 'scheduled_hour'] = 7
df_all.loc[(df_all['CRS_DEP_TIME'] >= 700) & (df_all['CRS_DEP_TIME'] < 800), 'scheduled_hour'] = 8
df_all.loc[(df_all['CRS_DEP_TIME'] >= 800) & (df_all['CRS_DEP_TIME'] < 900), 'scheduled_hour'] = 9
df_all.loc[(df_all['CRS_DEP_TIME'] >= 900) & (df_all['CRS_DEP_TIME'] < 1000), 'scheduled_hour'] = 10
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1000) & (df_all['CRS_DEP_TIME'] < 1100), 'scheduled_hour'] = 11
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1100) & (df_all['CRS_DEP_TIME'] < 1200), 'scheduled_hour'] = 12
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1200) & (df_all['CRS_DEP_TIME'] < 1300), 'scheduled_hour'] = 13
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1300) & (df_all['CRS_DEP_TIME'] < 1400), 'scheduled_hour'] = 14
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1400) & (df_all['CRS_DEP_TIME'] < 1500), 'scheduled_hour'] = 15
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1500) & (df_all['CRS_DEP_TIME'] < 1600), 'scheduled_hour'] = 16
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1600) & (df_all['CRS_DEP_TIME'] < 1700), 'scheduled_hour'] = 17
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1700) & (df_all['CRS_DEP_TIME'] < 1800), 'scheduled_hour'] = 18
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1800) & (df_all['CRS_DEP_TIME'] < 1900), 'scheduled_hour'] = 19
df_all.loc[(df_all['CRS_DEP_TIME'] >= 1900) & (df_all['CRS_DEP_TIME'] < 2000), 'scheduled_hour'] = 20
df_all.loc[(df_all['CRS_DEP_TIME'] >= 2000) & (df_all['CRS_DEP_TIME'] < 2100), 'scheduled_hour'] = 21
df_all.loc[(df_all['CRS_DEP_TIME'] >= 2100) & (df_all['CRS_DEP_TIME'] < 2200), 'scheduled_hour'] = 22
df_all.loc[(df_all['CRS_DEP_TIME'] >= 2200) & (df_all['CRS_DEP_TIME'] < 2300), 'scheduled_hour'] = 23
df_all.loc[(df_all['CRS_DEP_TIME'] >= 2300) & (df_all['CRS_DEP_TIME'] < 2400), 'scheduled_hour'] = 24

day = [1	,15	,14	,19	,17	,1	,17	,25	,13	,28	,17	,4	,3	,8	,31	,11	,12	,22	,23	,24	,25	,31	]
month = [1	,1	,2	,2	,3	,4	,4	,4	,5	,5	,6	,7	,9	,10	,10	,11	,11	,11	,11	,12	,12	,12	]
year = [2018, 2018, 2018, 2018, 2018, 2018	,2018,	2018,2018, 2018, 2018	,2018,	2018,2018, 2018, 2018	,2018,	2018,2018, 2018, 2018	,2018]
holidays = pd.DataFrame({'year': year,
                   'month': month,
                   'day': day
                   })

df_all['is_holiday'] = np.where((df_all['month'].isin(holidays.month)) & (df_all['day'].isin(holidays.day)),1,0)

def my_scaler(delay_df, from_col_name, to_col_name):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(pd.DataFrame(delay_df[from_col_name],index=delay_df.index))
    scaled_delay_df = pd.DataFrame(scaled, columns=['scale_temp'], index=delay_df.index)
    delay_df[to_col_name] = scaled_delay_df['scale_temp']
    
df_all['origin_canc_perc'] = df_all.groupby('ORIGIN')['CANC_DIV'].transform('mean')
my_scaler(df_all,'origin_canc_perc','scaled_origin_airport_score')

df_all['dest_canc_perc'] = df_all.groupby('DEST')['CANC_DIV'].transform('mean')
my_scaler(df_all,'dest_canc_perc','scaled_dest_airport_score')

df_all['airline_canc_perc'] = df_all.groupby('OP_CARRIER')['CANC_DIV'].transform('mean')
my_scaler(df_all,'airline_canc_perc','scaled_airline_score')

df_all['weekday_canc_prcnt'] = df_all.groupby('weekday')['CANC_DIV'].transform('mean')
my_scaler(df_all,'weekday_canc_prcnt','scaled_weekday_delay_score')

df_all['month_canc_prcnt'] = df_all.groupby('month')['CANC_DIV'].transform('mean')
my_scaler(df_all,'month_canc_prcnt','scaled_month_delay_score')


Define dataset for delays and calculate standard normally scaled scores

In [5]:
delay_df = df_all[(df_all['CANCELLED']==0) & (df_all['DIVERTED']==0)]

delay_df["ARRIVAL_DELAYED"] = np.where(delay_df['ARR_DELAY'] > 15, 1, 0)
delay_df['log_delay'] = np.where(delay_df['ARR_DELAY'] > 0, np.log1p(delay_df.ARR_DELAY) + 1, -1 * ( 1+ np.log1p(delay_df.ARR_DELAY * -1)))
    
delay_df['mean_origin_airport_delay'] = delay_df.groupby('ORIGIN')['ARR_DELAY'].transform('mean')
delay_df['airport_origin_delay_prcnt'] = delay_df.groupby('ORIGIN')['ARRIVAL_DELAYED'].transform('mean')
delay_df['airport_origin_delay_score'] = delay_df['mean_origin_airport_delay'] * delay_df['airport_origin_delay_prcnt']

my_scaler(delay_df,'airport_origin_delay_score','scaled_origin_airport_score')

delay_df['mean_dest_airport_delay'] = delay_df.groupby('DEST')['ARR_DELAY'].transform('mean')
delay_df['airport_dest_delay_prcnt'] = delay_df.groupby('DEST')['ARRIVAL_DELAYED'].transform('mean')
delay_df['airport_dest_delay_score'] = delay_df['mean_dest_airport_delay'] * delay_df['airport_dest_delay_prcnt']

my_scaler(delay_df,'airport_dest_delay_score','scaled_dest_airport_score')

delay_df['mean_airline_delay'] = delay_df.groupby('OP_CARRIER')['ARR_DELAY'].transform('mean')
delay_df['airline_delay_prcnt'] = delay_df.groupby('OP_CARRIER')['ARRIVAL_DELAYED'].transform('mean')
delay_df['airline_delay_score'] = delay_df['mean_airline_delay'] * delay_df['airline_delay_prcnt']
my_scaler(delay_df,'airline_delay_score','scaled_airline_score')

delay_df['mean_weekday_delay'] = delay_df.groupby('weekday')['ARR_DELAY'].transform('mean')
delay_df['weekday_delay_prcnt'] = delay_df.groupby('weekday')['ARRIVAL_DELAYED'].transform('mean')
delay_df['weekday_delay_score'] = delay_df['mean_weekday_delay'] * delay_df['weekday_delay_prcnt']
my_scaler(delay_df,'weekday_delay_prcnt','scaled_weekday_delay_score')

delay_df['mean_month_delay'] = delay_df.groupby('month')['ARR_DELAY'].transform('mean')
delay_df['month_delay_prcnt'] = delay_df.groupby('month')['ARRIVAL_DELAYED'].transform('mean')
delay_df['month_delay_score'] = delay_df['mean_month_delay'] * delay_df['month_delay_prcnt']
my_scaler(delay_df,'month_delay_prcnt','scaled_month_delay_score')

delay_df['mean_day_delay'] = delay_df.groupby('day')['ARR_DELAY'].transform('mean')
delay_df['day_delay_prcnt'] = delay_df.groupby('day')['ARRIVAL_DELAYED'].transform('mean')
delay_df['day_delay_score'] = delay_df['mean_day_delay'] * delay_df['day_delay_prcnt']
my_scaler(delay_df,'day_delay_prcnt','scaled_day_delay_score')

In [ ]:
#check
numberOfNullArrivalDelayWhenArrivalData = sum((df_all.CRS_ARR_TIME >0) & (df_all.ARR_TIME >0) & (df_all.ARR_DELAY.isnull()))
numberOfNoDataAndNotCancelledNorDiverted = sum(((delay_df.CANCELLED==0) & (delay_df.DIVERTED==0)) & (delay_df.ARR_TIME.isnull()))

print(f"Number of flights with no arrival delay data, but can be derived: {numberOfNullArrivalDelayWhenArrivalData}")
print(f"Number of flights that are missing data and were not cancelled, nor diverted: {numberOfNoDataAndNotCancelledNorDiverted}")

Explore datasets

In [ ]:
hours = df_all.scheduled_hour.value_counts()
months = df_all.month.value_counts()
weekdays = df_all.weekday.value_counts()
carriers = df_all.OP_CARRIER.unique()
origins = df_all.ORIGIN.unique()
destinations = df_all.DEST.unique()
cancelled = df_all.CANCELLED.value_counts()
div = df_all.DIVERTED.value_counts()

display(cancelled)
display(div)
display(months)
display(weekdays)
print(f"\nThe number of different carriers equals (in the train set): {len(carriers)}")
print(f"\nThe number of different destinations equals (in the train set): {len(destinations)}")
print(f"\nThe number of different origins equals (in the train set): {len(origins)}")

In [6]:
cat_var = ['OP_CARRIER','ORIGIN', 'DEST', 'scheduled_hour', 'month', 'is_holiday', 'weekday']

In [ ]:
for var in cat_var:
  df_all[var].value_counts().plot(figsize=(20, 10), kind="bar",title=f"Frequency distribution variable {var}")
  plt.show()

Split both datasets into train, validation and test sets

In [7]:
seed = 123
train_df, aux_df = train_test_split(delay_df, train_size=.5, random_state=seed)
validation_df, test_df = train_test_split(aux_df, train_size=.5, random_state=seed)

canc_train_df, canc_aux_df = train_test_split(df_all, train_size=.5, random_state=seed)
canc_validation_df, canc_test_df = train_test_split(canc_aux_df, train_size=.5, random_state=seed)

In [ ]:
train_df['ARR_DELAY'].describe(percentiles=[0.95,0.975, 0.98, 0.99, 0.995, 1.0])

In [8]:
#remove outliers
toppart_df = (train_df['ARR_DELAY'] > train_df['ARR_DELAY'].quantile(0.999))
train_df = train_df.loc[~toppart_df,:]

#remove infrequent airports and airlines
train_df['origin_count'] = train_df.groupby('ORIGIN')['ORIGIN'].transform('count')
train_df['dest_count'] = train_df.groupby('DEST')['DEST'].transform('count')
train_df['air_count'] = train_df.groupby('OP_CARRIER')['OP_CARRIER'].transform('count')

minimum_ori_df=train_df['origin_count'].quantile(0.05)
minimum_dest_df=train_df['dest_count'].quantile(0.05)
minimum_air_df=train_df['air_count'].quantile(0.05)

train_df = train_df[(((train_df['origin_count']>minimum_ori_df) & (train_df['dest_count']>minimum_dest_df)) & (train_df['air_count']>minimum_air_df))]

In [9]:
#remove infrequent airports and airlines
canc_train_df['origin_count'] = canc_train_df.groupby('ORIGIN')['ORIGIN'].transform('count')
canc_train_df['dest_count'] = canc_train_df.groupby('DEST')['DEST'].transform('count')
canc_train_df['air_count'] = canc_train_df.groupby('OP_CARRIER')['OP_CARRIER'].transform('count')

minimum_ori_df=canc_train_df['origin_count'].quantile(0.05)
minimum_dest_df=canc_train_df['dest_count'].quantile(0.05)
minimum_air_df=canc_train_df['air_count'].quantile(0.05)

canc_train_df = canc_train_df[(((canc_train_df['origin_count']>minimum_ori_df) & (canc_train_df['dest_count']>minimum_dest_df)) & (canc_train_df['air_count']>minimum_air_df))]

In [10]:
def plot_balancing(df, var):
  target_count = df[var].value_counts()
  target_count.plot(kind='bar', title=var);
  print(f"{len(df.loc[df[var] == 1]) / len(df)*100}%")


In [ ]:
plot_balancing(train_df, 'ARRIVAL_DELAYED')

In [ ]:
plot_balancing(canc_train_df, 'CANC_DIV')

In [11]:
final_factors = ['scaled_weekday_delay_score', 'scaled_month_delay_score', 'scaled_airline_score', 'scaled_origin_airport_score', 'scheduled_hour', 'scaled_dest_airport_score', 'is_holiday']

In [48]:
X_trained = train_df.loc[:,final_factors]
y_bin_train = train_df.loc[:,'ARRIVAL_DELAYED']
y_log_train = train_df.loc[:,'log_delay']
y_delay_train = train_df.loc[:,'ARR_DELAY']
X_val = validation_df.loc[:,final_factors]
y_bin_val = validation_df.loc[:,'ARRIVAL_DELAYED']
y_log_val = validation_df.loc[:,'log_delay']
y_delay_val = validation_df.loc[:,'ARR_DELAY']
X_test = test_df.loc[:,final_factors]
y_bin_test = test_df.loc[:,'ARRIVAL_DELAYED']
y_log_test = test_df.loc[:,'log_delay']
y_delay_test = test_df.loc[:,'ARR_DELAY']

In [13]:
final_canc_factors = ['scaled_weekday_delay_score', 'scaled_month_delay_score', 'scaled_airline_score', 'scaled_origin_airport_score', 'scheduled_hour', 'scaled_dest_airport_score', 'is_holiday']
X_trained_canc = canc_train_df.loc[:,final_canc_factors]
y_canc_trained = canc_train_df.loc[:,'CANC_DIV']

X_val_canc = canc_validation_df.loc[:,final_canc_factors]
y_canc_val = canc_validation_df.loc[:,'CANC_DIV']

X_test_canc = canc_test_df.loc[:,final_canc_factors]
y_canc_test = canc_test_df.loc[:,'CANC_DIV']

Explore independent factors

In [ ]:
delay_df.loc[:,final_factors].describe()

In [ ]:
df_all.loc[:,final_canc_factors].describe()

Balance Training Sets

In [14]:
#Balance training set
print('Resampled dataset shape %s' % Counter(y_bin_train))
smt = SMOTETomek(random_state=10)
X_train, y_train = smt.fit_resample(X_trained, y_bin_train)
print('Resampled dataset shape %s' % Counter(y_train))

Resampled dataset shape Counter({0: 121219, 1: 27411})
Resampled dataset shape Counter({0: 112795, 1: 112795})


In [15]:
#Balance training set
print('Resampled dataset shape %s' % Counter(y_canc_trained))
smt = SMOTETomek(random_state=10)
X_canc_train, y__canc_train = smt.fit_resample(X_trained_canc, y_canc_trained)
print('Resampled dataset shape %s' % Counter(y__canc_train))

Resampled dataset shape Counter({0: 148718, 1: 2878})
Resampled dataset shape Counter({0: 148222, 1: 148222})


See if datasets are balanced

In [ ]:
y_train.value_counts().plot(figsize=(20, 10), kind="bar",title=f"ARRIVAL_DELAYED")
plt.show()

In [ ]:
y__canc_train.value_counts().plot(figsize=(20, 10), kind="bar",title=f"ARRIVAL_DELAYED")
plt.show()

Visualize correlation matrices

In [ ]:
print(seaborn.heatmap(X_train.corr(method="pearson")) ) 

In [ ]:
print(seaborn.heatmap(X_canc_train.corr(method="pearson")))  


Visualize distribution of delays

In [ ]:
plt.figure(figsize=(15,5))
ax = train_df.loc[:,'ARR_DELAY'].plot.hist(bins=200)
ax.set_xlim(-50, 100)
plt.show()

Plot delays over time

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,5))

ax1.plot(train_df.groupby('DATES')['ARRIVAL_DELAYED'].mean(),data=train_df, color='g')
ax1.set_xlabel('Days of the year')
ax1.set_ylabel('late %', color='g')
ax1.set_xlim(min(train_df['DATES']), max(train_df['DATES']))

ax2 = ax1.twinx()
ax2.plot(train_df.groupby('DATES')['ARR_DELAY'].mean(),data=train_df, color = "r")
ax2.set_ylabel('Time of delay', color = "r")

Explore delays per category

In [ ]:
for var in cat_var:
    fig = plt.figure(figsize=(20,6))
    ax1 = fig.gca()

    train_df.groupby(var)['ARR_DELAY','ARRIVAL_DELAYED'].mean().sort_values(by='ARR_DELAY', ascending=False).plot(kind='bar', secondary_y='ARRIVAL_DELAYED', ax=ax1)
    plt.show()

Explore cancellations over time

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,5))

ax1.plot(canc_train_df.groupby('DATES')['CANC_DIV'].mean(),data=train_df, color='g')
ax1.set_xlabel('Days of the year')
ax1.set_ylabel('cancelled/diverted %', color='g')
ax1.set_xlim(min(train_df['DATES']), max(train_df['DATES']))

Explore cancellations per category

In [ ]:
for var in cat_var:
    fig = plt.figure(figsize=(20,6))
    ax1 = fig.gca()

    canc_train_df.groupby(var)['CANC_DIV'].mean().plot(kind='bar', ax=ax1)
    plt.show()

**Training and Evaluation**

1. Define Models for delays on training sets
2. Plot and evaluate results

In [51]:
#Tree based on cost_FP, cost_FN
ctree = tree.DecisionTreeClassifier()
rtree = tree.DecisionTreeRegressor()
model_bin_ctree = ctree.fit(X_train, y_train)
model_reg_tree = rtree.fit(X_trained, y_log_train)

#Random Forest
rfc = RandomForestClassifier()
rrfc = RandomForestRegressor()
model_bin_rfc = rfc.fit(X_train, y_train)
model_reg_rfc = rrfc.fit(X_trained, y_log_train)

#Bagged tree
clb = BaggingClassifier()
model_bin_clb = clb.fit(X_train, y_train)
rclb = BaggingRegressor()
model_reg_clb = rclb.fit(X_trained, y_log_train)

#Gradient Boosting
gb_clf = GradientBoostingClassifier()
model_bin_gb_clf = gb_clf.fit(X_train, y_train)
rgb_clf = GradientBoostingRegressor()
model_reg_gb_clb = rgb_clf.fit(X_trained, y_log_train)

Define functions to plot evaluation results

In [107]:
def plot_roc(y, hat_prob_y):
    RocCurveDisplay.from_predictions(
        y,
        hat_prob_y,
        name="ROC curve",
        color="darkorange",
    )
    plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    plt.axis("square")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC curve")
    plt.legend()
    plt.show()

def plot_confusion_matrix(hat_y, y, target_names):
    matrix = confusion_matrix(y, hat_y)  
    sns.heatmap(matrix.T, square=True, annot=True, fmt="d", cbar=False,
    xticklabels=target_names, yticklabels=target_names)
    plt.xlabel("true label")
    plt.ylabel("predicted label")

def estimate_cost(hat_y, y, cost_FP, cost_FN):
    return  np.sum(np.multiply(hat_y, (1 - y)) * cost_FP) + np.sum(np.multiply((1 - hat_y), y) * cost_FN)

def plot_to_evaluate(y, y_hat, title):
  print(f"{title} costs: {estimate_cost(y_hat, y, 1, 1)}")
  print(f"{title} MSE:{mean_squared_error(y, y_hat)}")
  print(f"{title} RMSE:{mean_squared_error(y, y_hat)**0.5}")
  print(f"report: {classification_report(y, y_hat)}")
  print(f"{title} accuracy score: {accuracy_score(y, y_hat)}")

def print_results(y, y_hat, title):
  print(f"{title} MAE:{mean_absolute_error(y, y_hat)}")
  print(f"{title} MSE:{mean_squared_error(y, y_hat)}")
  print(f"{title} RMSE:{mean_squared_error(y, y_hat)**0.5}")
  print(f"{title} Explained Variance Score:{explained_variance_score(y, y_hat)}")
  print(f"{title} R2: {r2_score(y, y_hat)}")

Plot and evaluate them on validation sets

In [ ]:
plot_to_evaluate(y_bin_val, model_bin_ctree.predict(X_val),  "Simple decision tree")
plot_to_evaluate(y_bin_val, model_bin_rfc.predict(X_val),"Random Forest")
plot_to_evaluate(y_bin_val, model_bin_clb.predict(X_val),  "Bagging Tree")
plot_to_evaluate(y_bin_val, model_bin_gb_clf.predict(X_val),  "Gradient Boosting")

print_results(y_log_val, model_reg_tree.predict(X_val), "Regression Tree")
print_results(y_log_val, model_reg_rfc.predict(X_val), "Random Forest")
print_results(y_log_val, model_reg_clb.predict(X_val), "Bagging regressor")
print_results(y_log_val, model_reg_gb_clb.predict(X_val), "Regression Boost")

In [ ]:
plot_confusion_matrix(y_bin_val, model_bin_ctree.predict(X_val), [0, 1])

In [ ]:
plot_confusion_matrix(y_bin_val, model_bin_rfc.predict(X_val), [0, 1])

In [ ]:
plot_confusion_matrix(y_bin_val, model_bin_clb.predict(X_val), [0, 1])

In [ ]:
plot_confusion_matrix(y_bin_val, model_bin_gb_clf.predict(X_val), [0, 1])

In [ ]:
probs1 = model_bin_ctree.predict_proba(X_val)[:,1]
precision1, recall1, _ = precision_recall_curve(y_bin_val, probs1)
probs2 = model_bin_rfc.predict_proba(X_val)[:,1]
precision2, recall2, _ = precision_recall_curve(y_bin_val, probs2)
probs3 = model_bin_clb.predict_proba(X_val)[:,1]
precision3, recall3, _ = precision_recall_curve(y_bin_val, probs3)
probs4 = model_bin_gb_clf.predict_proba(X_val)[:,1]
precision4, recall4, _ = precision_recall_curve(y_bin_val, probs4)
no_skill = len(y_bin_val[y_bin_val==1]) / len(y_bin_val)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall1, precision1, marker='.', label='Simple')
pyplot.plot(recall2, precision2, marker='.', label='RF')
pyplot.plot(recall3, precision3, marker='.', label='Bagged')
pyplot.plot(recall4, precision4, marker='.', label='Gradient')
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
pyplot.legend()
pyplot.show()

Develop models for cancelled/diverted flights based on training sets and train them on validation set

In [33]:
#Tree based on cost_FP, cost_FN
ctree = tree.DecisionTreeClassifier()
    # class_weight=class_weight)
model_canc_bin_ctree = ctree.fit(X_canc_train, y__canc_train)

#Random Forest
rfc = RandomForestClassifier()
# class_weight=class_weight)
model_canc_bin_rfc = rfc.fit(X_canc_train, y__canc_train)

#Bagged tree
clb = BaggingClassifier()
model_canc_bin_clb = clb.fit(X_canc_train, y__canc_train)

#Gradient Boosting
gb_clf = GradientBoostingClassifier()
model_canc_bin_gb_clf = gb_clf.fit(X_canc_train, y__canc_train)



Plot and evaluate

In [ ]:
plot_to_evaluate(y_canc_val, model_canc_bin_ctree.predict(X_val_canc), model_canc_bin_ctree.predict_proba(X_val_canc)[:,1], "Simple decision tree")
plot_to_evaluate(y_canc_val, model_canc_bin_rfc.predict(X_val_canc), model_canc_bin_rfc.predict_proba(X_val_canc)[:,1], "Random Forest")
plot_to_evaluate(y_canc_val, model_canc_bin_clb.predict(X_val_canc), model_canc_bin_clb.predict_proba(X_val_canc)[:,1], "Bagging Tree")
plot_to_evaluate(y_canc_val, model_canc_bin_gb_clf.predict(X_val_canc), model_canc_bin_gb_clf.predict_proba(X_val_canc)[:,1], "Gradient Boosting")

In [ ]:
plot_confusion_matrix(y_canc_val, model_canc_bin_ctree.predict(X_val_canc), [0, 1])

In [ ]:
plot_confusion_matrix(y_canc_val, model_canc_bin_rfc.predict(X_val_canc), [0, 1])

In [ ]:
plot_confusion_matrix(y_canc_val, model_canc_bin_clb.predict(X_val_canc), [0, 1])

In [ ]:
plot_confusion_matrix(y_canc_val, model_canc_bin_gb_clf.predict(X_val_canc), [0, 1])

In [ ]:
probs1 = model_canc_bin_ctree.predict_proba(X_val_canc)[:,1]
precision1, recall1, _ = precision_recall_curve(y_canc_val, probs1)
probs2 = model_canc_bin_rfc.predict_proba(X_val_canc)[:,1]
precision2, recall2, _ = precision_recall_curve(y_canc_val, probs2)
probs3 = model_canc_bin_clb.predict_proba(X_val_canc)[:,1]
precision3, recall3, _ = precision_recall_curve(y_canc_val, probs3)
probs4 = model_canc_bin_gb_clf.predict_proba(X_val_canc)[:,1]
precision4, recall4, _ = precision_recall_curve(y_canc_val, probs4)
no_skill = len(y_canc_val[y_canc_val==1]) / len(y_canc_val)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall1, precision1, marker='.', label='Simple')
pyplot.plot(recall2, precision2, marker='.', label='RF')
pyplot.plot(recall3, precision3, marker='.', label='Bagged')
pyplot.plot(recall4, precision4, marker='.', label='Gradient')
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
pyplot.legend()
pyplot.show()

**Training models**

1. Train each model using a 5-fold cross validation
2. Retrieve the best parameters
3. Plot the feature importances

In [53]:
learning_rates = [0.01, 0.03, 0.05, 0.07, 0.09, 0.1]
max_depths = [3,4,5,6,7,8]
min_samples_splits = [3,4,5,6,7,8]
min_samples_leafs = [1, 2, 3,4,5,6,7,8]
criterions = ['gini', 'entropy']

In [ ]:
cost_FP = 100
cost_FN = 18
param_grid = {
    "criterion": criterions,
    "max_depth": max_depths,
    "min_samples_split": min_samples_splits,
    "min_samples_leaf": min_samples_leafs,
}
def cost_fun(y, hat_y):
  return np.sum(np.multiply(hat_y, (1 - y)) * cost_FP) + np.sum(np.multiply((1 - hat_y), y) * cost_FN)

basemodel = tree.DecisionTreeClassifier()
tuned_delayed_model = HalvingGridSearchCV(basemodel, param_grid, scoring=make_scorer(cost_fun, greater_is_better=True), n_jobs=-1, min_resources="smallest", factor=6, cv=5)
tuned_delayed_model.fit(X_train, y_train)
tuned_delayed_model.best_params_

In [ ]:
importances = tuned_delayed_model.best_estimator_.feature_importances_
# summarize feature importance
for i,v in enumerate(importances):
	print(f"{i} {X_val.columns[i]} Score: {v}")
# plot feature importance
pyplot.bar([x for x in range(len(importances))], importances)
pyplot.show()

In [ ]:
cost_FP = 100
cost_FN = 2
def cost_fun(y, hat_y):
  return np.sum(np.multiply(hat_y, (1 - y)) * cost_FP) + np.sum(np.multiply((1 - hat_y), y) * cost_FN)

basemodel = tree.DecisionTreeClassifier()
tuned_canc_model = HalvingGridSearchCV(basemodel, param_grid, scoring=make_scorer(cost_fun, greater_is_better=True), n_jobs=-1, min_resources="smallest", factor=6, cv=5)
tuned_canc_model.fit(X_canc_train, y__canc_train)
tuned_canc_model.best_params_

In [ ]:
importances = tuned_canc_model.best_estimator_.feature_importances_
# summarize feature importance
for i,v in enumerate(importances):
	print(f"{i} {X_val.columns[i]} Score: {v}")
# plot feature importance
pyplot.bar([x for x in range(len(importances))], importances)
pyplot.show()

In [134]:
GBR=GradientBoostingRegressor()
search_grid={'n_estimators':[10, 50, 100],'learning_rate':learning_rates, 'max_depth':max_depths, 'random_state':[10]}
tuned_log_delay_model=HalvingGridSearchCV(estimator=GBR,param_grid=search_grid,scoring='neg_mean_squared_error',n_jobs=-1, cv=5)

In [ ]:
tuned_log_delay_model.fit(X_val,y_log_val)
print(f"best parameters: {tuned_log_delay_model.best_params_}")

In [ ]:
importances = tuned_log_delay_model.best_estimator_.feature_importances_
# summarize feature importance
for i,v in enumerate(importances):
	print(f"{i} {X_val.columns[i]} Score: {v}")
# plot feature importance
pyplot.bar([x for x in range(len(importances))], importances)
pyplot.show()

**Predictions**

1. Fit the optimal hyperparameters for each model
2. Predict them on the test set
3. Evaluate them on the test set
4. Plot them over time

In [ ]:
tuned_delayed_model.best_estimator_.fit(X_train, y_train)
pred_delay_prob = tuned_delayed_model.best_estimator_.predict(X_test)

In [ ]:
data = {'date': test_df['DATES'], 'predicted': pred_delay_prob, 'actual': y_bin_test}
delayed_results = pd.DataFrame(data=data)
delayed_results['delta'] = delayed_results['actual'] - delayed_results['predicted']

In [ ]:
plot_to_evaluate(y_bin_test, pred_delay_prob, "Delay Probability evaluation on test set")

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,5))

ax1.plot(delayed_results.groupby('date')['actual'].mean(),data=delayed_results, color='g')
ax1.plot(delayed_results.groupby('date')['predicted'].mean(),data=delayed_results, color='b')
ax1.set_xlabel('Days of the year')
ax1.set_ylabel('Fraction of delays', color='g')
ax1.set_xlim(min(delayed_results['date']), max(delayed_results['date']))

In [ ]:
tuned_canc_model.best_estimator_.fit(X_canc_train, y__canc_train)
pred_canc_prob = tuned_canc_model.predict(X_test_canc)

In [ ]:
data = {'date': canc_test_df['DATES'], 'predicted': pred_canc_prob, 'actual': y_canc_test}
canc_results = pd.DataFrame(data=data)
canc_results['delta'] = canc_results['actual'] - canc_results['predicted']

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,5))

ax1.plot(canc_results.groupby('date')['actual'].mean(),data=canc_results, color='g')
ax1.plot(canc_results.groupby('date')['predicted'].mean(),data=canc_results, color='b')
ax1.set_xlabel('Days of the year')
ax1.set_ylabel('Fraction of cancelled/diverted flights', color='g')
ax1.set_xlim(min(canc_results['date']), max(canc_results['date']))

In [ ]:
plot_to_evaluate(y_canc_test, pred_canc_prob, "Cancelled/Diverted Flight Probability evaluation on test set")

In [ ]:
reg_model = GradientBoostingRegressor(
    max_depth=best_params_reg['max_depth'],
    learning_rate= best_params_reg['learning_rate'],
    n_estimators= best_params_reg['n_estimators'],

)
reg_model.fit(X_trained, y_log_train)
pred_delay = reg_model.predict(X_test)
print(f"{mean_squared_error(y_log_test, pred_delay)**0.5}")
print(f"{explained_variance_score(y_log_test, pred_delay)}")
print(f"{estimate_cost(pred_delay, y_log_test, 100, 1)}")


In [105]:
data = {'date': test_df['DATES'], 'predicted': pred_delay, 'actual': y_log_test}
log_delay_results = pd.DataFrame(data=data)
log_delay_results['delta'] = log_delay_results['actual'] - log_delay_results['predicted']

In [ ]:
fig, ax1 = plt.subplots(figsize=(30,5))

ax1.plot(log_delay_results.groupby('date')['actual'].mean(),data=log_delay_results, color='g')
ax1.plot(log_delay_results.groupby('date')['predicted'].mean(),data=log_delay_results, color='b')
ax1.set_xlabel('Days of the year')
ax1.set_ylabel('log delays', color='g')
ax1.set_xlim(min(log_delay_results['date']), max(log_delay_results['date']))